In [77]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import math
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import datasets
from random import *

np.set_printoptions(threshold=np.nan)

# Datasets definition
# -------------------------------------------------------------------------------------------------------
digits = datasets.load_digits()
X_digits = digits.data
Y_digits = digits.target

indAll = np.arange(len(Y_digits))
np.random.shuffle(indAll)

testToTrainRatio = 0.8
trainX=X_digits[0:int(testToTrainRatio*len(indAll))]
testX=X_digits[int(testToTrainRatio*len(indAll)):]
trainY=Y_digits[0:int(testToTrainRatio*len(indAll))]
testY=Y_digits[int(testToTrainRatio*len(indAll)):]

trainingDataset=X_digits
target=Y_digits
testingDataset=X_digits
testingTarget=Y_digits

target = [[target[i] for j in range(1)] for i in range(len(target))]
target=np.asarray(target)
print("target.shape: "+str(target.shape))
 
numberOfLayerNodes=[64,32,1]
learningCoeffitient=0.01

weights = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes),np.amax(numberOfLayerNodes)))
nodes = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
weightedSums = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
errors = np.zeros((len(numberOfLayerNodes), np.amax(numberOfLayerNodes)))
rowError = 0
epochError = 0
currentDatasetRow = 0
                 
    
def populateWeightsWithRandom():
    global weights
    for z in range(1, len(numberOfLayerNodes)):
        for y in range(0, len(weights[0])):
            for x in range(0, len(weights[0][0])):
                weights[z][y][x] = random() * 2 - 1    
    
    
def calculateHiddenLayer(layerNumber):
    ws = 0
    for x in range(0,numberOfLayerNodes[layerNumber]):
        ws = 0
        for px in range(0, numberOfLayerNodes[layerNumber - 1]):
            ws = ws + nodes[layerNumber - 1][px] * weights[layerNumber][x][px]
            weightedSums[layerNumber][x] = ws
            nodes[layerNumber][x] = activationFunction(ws)

def activationFunction(ws):
    value = 0
    value = 1 / (1 + math.exp(-ws))
    return value                
       

def derivative(value):
    value = activationFunction(value) - math.pow(activationFunction(value), 2)
    return value
    
    
def calculateOutput():
    ws = 0
    sumOutputs = 0
    layerNumber = len(numberOfLayerNodes)-1
    for x in range(0, numberOfLayerNodes[layerNumber]):
        ws = 0
        for px in range(0, numberOfLayerNodes[layerNumber - 1]):
            ws = ws + nodes[layerNumber - 1][px] * weights[layerNumber][x][px]

        nodes[layerNumber][x] = ws
        weightedSums[layerNumber][x] = ws
        sumOutputs = sumOutputs + math.exp(nodes[layerNumber][x])

    for x in range(0, numberOfLayerNodes[layerNumber]):
        nodes[layerNumber][x] = math.exp(nodes[layerNumber][x]) / sumOutputs

            
def calculateErrors():
    rowError=0
    layerNumber = len(numberOfLayerNodes)-1
    for x in range(0, numberOfLayerNodes[layerNumber]):
#         print("-------------------------------------------- Debug")
#         print("layerNumber: "+str(layerNumber))
#         print("x: "+ str(x))
#         print("current dataset row: "+ str(x))
#         print("Errors ")
#         print(errors)
        errors[layerNumber][x] = derivative(weightedSums[layerNumber][x])* (target[currentDatasetRow][x] - nodes[layerNumber][x])
        rowError=rowError+math.sqrt(math.pow(target[currentDatasetRow][x] - nodes[layerNumber][x],2))

    rowError=rowError/numberOfLayerNodes[layerNumber]

    for layerNumber in range(len(numberOfLayerNodes)-2, 0, -1):
        for x in range(0, numberOfLayerNodes[layerNumber]):
            errors[layerNumber][x] = 0
            for nx in range(0, numberOfLayerNodes[layerNumber + 1]):
                errors[layerNumber][x] = errors[layerNumber + 1][nx] * weights[layerNumber + 1][nx][x]
            errors[layerNumber][x] = derivative(weightedSums[layerNumber][x]) * errors[layerNumber][x]

    
def backPropagate():
    for layerNumber in range(1, len(numberOfLayerNodes)):
        for x in range (0, numberOfLayerNodes[layerNumber]):
            for px in range(0, numberOfLayerNodes[layerNumber - 1]):
                weights[layerNumber][x][px] = weights[layerNumber][x][px] + learningCoeffitient * errors[layerNumber][x] * nodes[layerNumber - 1][px];
    

    
def training(nEpochs):
    global currentDatasetRow
    epochError=0;
    for epoch in range(0, nEpochs):
        for y in range(0, len(trainingDataset)):
            currentDatasetRow = y
            nodes[0] = trainingDataset[y]
            for layerNumber in range(1, len(numberOfLayerNodes)-1):
                calculateHiddenLayer(layerNumber)

            calculateOutput()
            calculateErrors()
            epochError=epochError+rowError
            backPropagate()

        epochError=epochError/nEpochs
        print("Error on epoch: "+str(epoch)+" = "+str(epochError))
    
    
    
def classify(row):
    nodes[0] = row
    for layerNumber in range(1, len(numberOfLayerNodes)):
        calculateHiddenLayer(layerNumber)

    calculateOutput()
    
    
    
print("-----------------------")
print(weights.shape)
print(nodes.shape)
print(weightedSums.shape)
print(errors.shape)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

populateWeightsWithRandom()
# print(weights)
calculateHiddenLayer(2)

training(10)
    
    


target.shape: (1797, 1)
-----------------------
(3, 64, 64)
(3, 64)
(3, 64)
(3, 64)
(1437, 64)
(1437,)
(360, 64)
(360,)
Error on epoch: 0 = 0.0
Error on epoch: 1 = 0.0
Error on epoch: 2 = 0.0
Error on epoch: 3 = 0.0
Error on epoch: 4 = 0.0
Error on epoch: 5 = 0.0
Error on epoch: 6 = 0.0
Error on epoch: 7 = 0.0
Error on epoch: 8 = 0.0
Error on epoch: 9 = 0.0


In [85]:
%time
yyy=0
for x in range(10000):
    yyy=yyy+1
    
print(yyy)
%time

Wall time: 0 ns
10000
Wall time: 0 ns


In [86]:
class timeit():
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

In [94]:
with timeit():
    y=0
    for x in range(100000000):
        y=y+1

runtime: 0:00:10.922140


In [96]:
y=0
for x in range(100000000):
    y=y+1
print(y)

100000000


In [79]:
%timeit